<a href="https://colab.research.google.com/github/kghilothia06/Time-Series-Forecasting/blob/main/05_grid_search_EXPONENTIAL_SMOOTHING_car_sales_12_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Grid Search Framework for ETS models for sales prediction

Import Libraries

In [1]:
from math import sqrt
from multiprocessing import cpu_count
from joblib import Parallel
from joblib import delayed
from warnings import catch_warnings
from warnings import filterwarnings
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_squared_error
from pandas import read_excel
from numpy import array
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


one-step Holt Winter’s Exponential Smoothing forecast

In [2]:
def exp_smoothing_forecast(history, config):
    t,d,s,p,b,r = config
    # define model
    history = array(history)
    model = ExponentialSmoothing(history, trend=t, damped=d, seasonal=s, seasonal_periods=p)
    # fit model
    model_fit = model.fit(optimized=True, use_boxcox=b, remove_bias=r)
    # make one step forecast
    yhat = model_fit.predict(len(history), len(history))
    return yhat[0]

walk-forward validation for univariate data

In [3]:
def walk_forward_validation(data, n_test, cfg):
    predictions = list()
    # split dataset
    train = data[:-n_test]
    test = data[-n_test:]
    # seed history with training dataset
    history = [x for x in train]
    # step over each time-step in the test set
    for i in range(len(test)):
        # fit model and make forecast for history
        yhat = exp_smoothing_forecast(history, cfg)
        # store forecast in list of predictions
        predictions.append(yhat)
        # add actual observation to history for the next loop
        history.append(test[i])
    # estimate prediction error
    error = sqrt(mean_squared_error(test, predictions))
    print('RMSE : ' + str(error))


Load dataset for monthly sales prediction

In [4]:
# load dataset
series = read_excel('/content/Sales-monthly.xlsx', header=0, index_col=0)
data = series.values
# data split
n_test = 12

specify model configs

In [5]:
t = None
d =False
s = 'add'
p = 12
b = True
r = False
cfg = [t,d,s,p,b,r]

In [6]:
walk_forward_validation(data,n_test,cfg)

RMSE : 19436.147658159283


Load dataset for quarterly sales prediction

In [7]:
# load dataset
series = read_excel('/content/Sales-quarterly.xlsx', header=0, index_col=0)
series = pd.DataFrame(series['Sales'])
data = series.values
# data split
n_test = 4

specify model configs

In [8]:
t = None
d = False
s = 'add'
p = 12
b = False
r = True
cfg = [t,d,s,p,b,r]

In [9]:
walk_forward_validation(data,n_test,cfg)

RMSE : 71793.47467841941
